In [2]:
# %pip install -U openai pandas tqdm

import os
import json
import time
import random
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from tqdm.auto import tqdm
from openai import OpenAI


In [6]:
os.environ["OPENAI_API_KEY"] = "<APIKEY>"

client = OpenAI()

Note: Prompt Template 2 was used for the final classification

In [77]:
PROMPT_TEMPLATE_1 = r"""
You are classifying a Bluesky user into exactly ONE of these 5 categories:
- Right
- Center-Left
- Progressive-Left
- Extreme-Left
- Unknown

You must be conservative. If evidence is insufficient, ambiguous, mixed, or mostly non-political, choose "Unknown".
Use ONLY the text provided (bio + last 14 days of posts). Do NOT guess demographics, location, identity, or intent.

-------------------------
LEFT-BUCKET FRAMEWORK (use these 3 dimensions)
1) Capitalism:
   - Center-Left: reform capitalism (regulated markets, safety nets, incremental reform)
   - Progressive-Left: replace capitalism with more democratic/socialist systems (strong systemic critique, but still often within democratic norms)
   - Extreme-Left: anti-capitalism is a consistent, non-negotiable identity marker; reform is explicitly and repeatedly dismissed as complicit or insufficient

2) Institutions (liberal-democratic institutions like elections, courts, mainstream parties, incremental policy process):
   - Center-Left: works within institutions; defends legitimacy
   - Progressive-Left: critical of institutions but still engages strategically
   - Extreme-Left: consistently frames mainstream institutions as irredeemable; electoral engagement is absent or actively discouraged across posts
   
3) Strategy:
   - Center-Left: policy reforms, coalition building, voting/legislation focus
   - Progressive-Left: mass movements + systemic change, strong organizing/activism rhetoric. May use strong systemic or abolitionist rhetoric but still references electoral politics, public policy, or democratic processes as viable tools.
   - Extreme-Left: extra-institutional struggle as primary sustained framing; "burn it down" is a pattern not a vent; note that abolitionist slogans and hyperbolic frustration are common even among Progressive-Left users on social media and should not alone trigger this label


-------------------------
OPERATIONAL DEFINITION FOR "Right"
Label "Right" only if there are CLEAR, REPEATED, and ENDORSED signals of right-leaning ideology in the user’s own voice.

Examples of qualifying Right signals (must be endorsed, not merely mentioned):
- Pro-conservative or pro-right movement alignment (explicit self-ID or consistent advocacy)
- Pro-market / anti-socialist framing as a core theme (e.g., “socialism is the problem”, “welfare ruins society”, “unions are harmful”)
- Hostility to left movements framed approvingly (e.g., anti-BLM/anti-trans rights rhetoric framed as support for conservative policy)
- Nationalist or culturally conservative positions as recurring stances
- Anti-regulation / anti-tax / anti-government social spending as a recurring stance

Do NOT label Right if:
- The user is quoting opponents, dunking on right-wing views, or using sarcasm
- There are only weak signals (one post, vague “anti-woke” joke, edgy meme) without consistent stance
- The content is mostly apolitical or unclear

-------------------------
WATCH-OUTS (must follow)
- Sarcasm/irony: do not treat provocative phrases as endorsement unless the stance is clearly positive and consistent.
- Quoting: if the user quotes text, classify based on their commentary, not the quoted content.
- Do not classify based on one intense post; look for a pattern across multiple posts or an explicit bio.
- Mixed ideology: if conflicting signals and hard to resolve, choose Unknown.
- Sparse data: if too few posts or mostly non-political, choose Unknown.
- Social media compression: abolitionist or revolutionary language online is often expressive rather than programmatic. Weight patterns and consistency over isolated striking statements.

-------------------------
DECISION PROCESS (mandatory)
Step A) Political-signal gate:
- If bio+posts do not contain enough political content OR stance is too ambiguous/mixed OR data is sparse -> output Unknown.
- Political content must include clear normative positioning on public policy, institutions, or political actors. Mere reporting, aggregation, identity markers, aesthetic affiliation, or anti-politician insults without policy positioning are not sufficient.
- Accounts that primarily curate, monitor, or report political information without expressing a stance should be labeled Unknown.

Step B) If political signal is sufficient:
- Choose the best category:
  - If clearly Right (per definition above) → Right
  - Else classify into Center-Left / Progressive-Left / Extreme-Left using the 3-dimension framework.

-------------------------
CONFIDENCE (integer 1–10)
Confidence must reflect BOTH evidence quality and quantity:
- 9–10: many posts + explicit stance + consistent pattern + low ambiguity
- 7–8: clear pattern but less volume OR minor ambiguity
- 5–6: weak pattern, mixed signals, or interpretation-heavy → prefer Unknown
- 1–4: very little text, mostly non-political, or high ambiguity → MUST be Unknown

-------------------------
OUTPUT FORMAT (STRICT)
Return ONLY a JSON object exactly like:
{{"tag":"<Right|Center-Left|Progressive-Left|Extreme-Left|Unknown>","confidence":<1-10>}}

No extra keys.
No explanation text.
No markdown.
No trailing characters.

-------------------------
User bio:
<<<BIO
{bio}
BIO>>>

User posts (last 14 days, consolidated):
<<<POSTS
{posts}
POSTS>>>
""".strip()

The prompt below was used

In [88]:
PROMPT_TEMPLATE_2 = r"""
You are classifying a Bluesky user into exactly ONE of these 5 categories:
- Right
- Center-Left
- Progressive-Left
- Extreme-Left
- Unknown

You must be conservative. If evidence is insufficient, ambiguous, mixed, or mostly non-political, choose "Unknown".
Use ONLY the text provided (bio + last 14 days of posts). Do NOT guess demographics, location, identity, or intent.

-------------------------
LEFT-BUCKET FRAMEWORK (use these 3 dimensions)
1) Capitalism:
   - Center-Left: reform capitalism (regulated markets, safety nets, incremental reform)
   - Progressive-Left: Advocates major structural redistribution, democratic socialism, wealth restructuring, or systemic economic transformation beyond incremental reform. May frame oligarchy, wealth concentration, or corporate power as core structural problems, even if "abolish capitalism" language is not explicitly used.
   - Extreme-Left: anti-capitalism is a consistent, non-negotiable identity marker; reform is explicitly and repeatedly dismissed as complicit or insufficient

2) Institutions (liberal-democratic institutions like elections, courts, mainstream parties, incremental policy process):
   - Center-Left: works within institutions; defends legitimacy
   - Progressive-Left: critical of institutions but still engages strategically
   - Extreme-Left: consistently frames mainstream institutions as irredeemable; electoral engagement is absent or actively discouraged across posts
   
3) Strategy:
   - Center-Left: policy reforms, coalition building, voting/legislation focus
   - Progressive-Left: mass movements + systemic change, strong organizing/activism rhetoric. Advocates expansive redistributive or structural policies (e.g., universal healthcare, free public college, wealth taxes, labor expansion) framed as necessary systemic change rather than incremental adjustment. May use strong systemic or abolitionist rhetoric but still references electoral politics, public policy, or democratic processes as viable tools.
   - Extreme-Left: extra-institutional struggle as primary sustained framing; "burn it down" is a pattern not a vent; note that abolitionist slogans and hyperbolic frustration are common even among Progressive-Left users on social media and should not alone trigger this label


-------------------------
OPERATIONAL DEFINITION FOR "Right"
Label "Right" only if there are CLEAR, REPEATED, and ENDORSED signals of right-leaning ideology in the user’s own voice.

Examples of qualifying Right signals (must be endorsed, not merely mentioned):
- Pro-conservative or pro-right movement alignment (explicit self-ID or consistent advocacy)
- Pro-market / anti-socialist framing as a core theme (e.g., “socialism is the problem”, “welfare ruins society”, “unions are harmful”)
- Hostility to left movements framed approvingly (e.g., anti-BLM/anti-trans rights rhetoric framed as support for conservative policy)
- Nationalist or culturally conservative positions as recurring stances
- Anti-regulation / anti-tax / anti-government social spending as a recurring stance

Do NOT label Right if:
- The user is quoting opponents, dunking on right-wing views, or using sarcasm
- There are only weak signals (one post, vague “anti-woke” joke, edgy meme) without consistent stance
- The content is mostly apolitical or unclear

-------------------------
WATCH-OUTS (must follow)
- Sarcasm/irony: do not treat provocative phrases as endorsement unless the stance is clearly positive and consistent.
- Quoting: if the user quotes text, classify based on their commentary, not the quoted content.
- Do not classify based on one intense post; look for a pattern across multiple posts or an explicit bio.
- Mixed ideology: if conflicting signals and hard to resolve, choose Unknown.
- Sparse data: if too few posts or mostly non-political, choose Unknown.
- Social media compression: abolitionist or revolutionary language online is often expressive rather than programmatic. Weight patterns and consistency over isolated striking statements.

-------------------------
DECISION PROCESS (mandatory)
Step A) Political-signal gate:
- If bio+posts do not contain enough political content OR stance is too ambiguous/mixed OR data is sparse -> output Unknown.
- Political content must include clear normative positioning on public policy, governance, institutions, or political actors.
- Repeated systemic critique of parties, governance structures, or power systems counts as political positioning even if explicit policy proposals are absent.
- Mere reporting, aggregation, neutral civic language, identity markers, aesthetic affiliation, or generic anti-politician insults without structural or normative positioning are not sufficient.
- If an account explicitly describes itself as nonpartisan, neutral, or purely informational AND posts are primarily descriptive rather than evaluative, label as Unknown unless strong contrary evidence appears.

Step B) If political signal is sufficient:
- Choose the best category:
  - If clearly Right (per definition above) → Right
  - Else classify into Center-Left / Progressive-Left / Extreme-Left using the 3-dimension framework.

-------------------------
CONFIDENCE (integer 1–10)
Confidence must reflect BOTH evidence quality and quantity:
- 9–10: many posts + explicit stance + consistent pattern + low ambiguity
- 7–8: clear pattern but less volume OR minor ambiguity
- 5–6: weak pattern, mixed signals, or interpretation-heavy → prefer Unknown
- 1–4: very little text, mostly non-political, or high ambiguity → MUST be Unknown

-------------------------
OUTPUT FORMAT (STRICT)
Return ONLY a JSON object exactly like:
{{"tag":"<Right|Center-Left|Progressive-Left|Extreme-Left|Unknown>","confidence":<1-10>}}

No extra keys.
No explanation text.
No markdown.
No trailing characters.

-------------------------
User bio:
<<<BIO
{bio}
BIO>>>

User posts (last 14 days, consolidated):
<<<POSTS
{posts}
POSTS>>>
""".strip()

In [89]:
MODEL = "gpt-5.2"

BIO_COL = "users_bio"
POSTS_COL = "posts_last_2_weeks"

TAG_COL = "ideology_tag"
CONF_COL = "ideology_confidence"


In [90]:
FORMAT = {
    "type": "json_schema",
    "name": "ideology_classification",
    "strict": True,
    "schema": {
        "type": "object",
        "additionalProperties": False,
        "properties": {
            "tag": {
                "type": "string",
                "enum": ["Right", "Center-Left", "Progressive-Left", "Extreme-Left", "Unknown"]
            },
            "confidence": {
                "type": "integer",
                "minimum": 1,
                "maximum": 10
            }
        },
        "required": ["tag", "confidence"]
    }
}

In [91]:
# CSV taken from the drive
data = pd.read_csv('feed_likers_user_data.csv')
data.head()

,feed_url,feed_at_uri,creator_did,creator_handle,creator_display_name,liked_at,like_indexed_at,users_did,users_handle,users_display_name,users_bio,users_followers_count,users_follows_count,users_posts_count,posts_last_2_weeks,posts_last_2_weeks_count
0,https://bsky.app/profile/davidsacerdote.bsky.s...,at://did:plc:o4s55v3tsfph6whswxccpsia/app.bsky...,did:plc:o4s55v3tsfph6whswxccpsia,davidsacerdote.bsky.social,David Sacerdote,2026-02-24T01:55:37.344Z,2026-02-24T01:55:37.344Z,did:plc:k4cbzvzqpg4y4en4zfblcgpo,nathanrichards05.bsky.social,Nathan Richards,Wilmington\nPhotographer/Videographer\nFan of ...,2,40,2,Midway Games needs to be revived || They are w...,2
1,https://bsky.app/profile/davidsacerdote.bsky.s...,at://did:plc:o4s55v3tsfph6whswxccpsia/app.bsky...,did:plc:o4s55v3tsfph6whswxccpsia,davidsacerdote.bsky.social,David Sacerdote,2026-02-18T02:04:33.731Z,2026-02-18T02:04:33.731Z,did:plc:2d3v2xiczxzshcu66wdueuqq,steveohio.com,Steve.,Liberal AF\n> SteveOhio.com for links,202,233,36,I cannot in good conscience lend legitimacy to...,13
2,https://bsky.app/profile/davidsacerdote.bsky.s...,at://did:plc:o4s55v3tsfph6whswxccpsia/app.bsky...,did:plc:o4s55v3tsfph6whswxccpsia,davidsacerdote.bsky.social,David Sacerdote,2026-02-15T13:43:55.078Z,2026-02-15T13:43:55.078Z,did:plc:yfju5tg4fuwd7udc5uwu3pkh,corsau.bsky.social,Cor,I do solemnly swear that I will support and de...,198,1119,4,NaN,0
3,https://bsky.app/profile/davidsacerdote.bsky.s...,at://did:plc:o4s55v3tsfph6whswxccpsia/app.bsky...,did:plc:o4s55v3tsfph6whswxccpsia,davidsacerdote.bsky.social,David Sacerdote,2026-01-29T03:13:23.627Z,2026-01-29T03:13:23.627Z,did:plc:cii4hsxsnfnwkpmbl4clxq2l,guidry36.bsky.social,NaN,NaN,787,861,125,People are asking what else they can do.\n\nMy...,85
4,https://bsky.app/profile/davidsacerdote.bsky.s...,at://did:plc:o4s55v3tsfph6whswxccpsia/app.bsky...,did:plc:o4s55v3tsfph6whswxccpsia,davidsacerdote.bsky.social,David Sacerdote,2026-01-25T19:40:41.078Z,2026-01-25T19:40:41.078Z,did:plc:fcrqiuggptn3ncbyffxlti5j,biancatempt.bsky.social,Jesus drank.,"Jesus drank, then hurled the glass at the warp...",413,503,2390,"My brain contextualized ""thread"" to ""dread"". |...",82


In [97]:
df = data[data['posts_last_2_weeks_count'] >= 5]
df.shape

(1110, 16)

In [95]:
UNKNOWN_THRESHOLD = 6 

def classify_user(bio, posts, retries=3):
    bio = "" if pd.isna(bio) else str(bio)
    posts = "" if pd.isna(posts) else str(posts)
    prompt = PROMPT_TEMPLATE_2.format(bio=bio, posts=posts)

    last_err = None
    for k in range(retries):
        try:
            resp = client.responses.create(
                model=MODEL,
                input=[{"role": "user", "content": prompt}],
                text={"format": FORMAT},
                reasoning={"effort": "low"},
            )

            raw = resp.output_text
            if not raw:
                raise ValueError("Empty output_text")

            out = json.loads(raw)

            if out["confidence"] <= UNKNOWN_THRESHOLD:
                out["tag"] = "Unknown"

            return out

        except Exception as e:
            last_err = e
            time.sleep(1.5 * (k + 1))
            
    return {"tag": "Unknown", "confidence": 1}

In [96]:
def run_classification(df):
    tags, confs = [], []
    failures = 0

    for row in tqdm(df.itertuples(index=False), total=len(df)):
        try:
            res = classify_user(getattr(row, BIO_COL), getattr(row, POSTS_COL))
            tags.append(res["tag"])
            confs.append(res["confidence"])
        except Exception:
            failures += 1
            tags.append("Unknown")
            confs.append(1)

    print(f"Failures: {failures}/{len(df)}")

    df[TAG_COL] = tags
    df[CONF_COL] = confs
    return df

In [98]:
output_1000 = run_classification(df)

  0%|          | 0/1110 [00:00<?, ?it/s]

Failures: 0/1110


In [99]:
output_1000['ideology_tag'].value_counts()

ideology_tag
Unknown             411
Progressive-Left    380
Center-Left         302
Extreme-Left         14
Right                 3
Name: count, dtype: int64

In [100]:
output_1000.to_csv('user_classification_1000.csv',index=False)

In [ ]:
## Sample Evaluation

In [93]:
# dfsample = df.sample(30, random_state = 12)
# dfsample.head()

In [94]:
# dfsample = dfsample.reset_index()

In [85]:
evaluation_users = [
    "Kettering Foundation",
    "William the Mad",
    "WOKE_AI_GOD",
    "💙🌊 Winn 🌊💙",
    "MarieGuarde",
    "Jeff/傑夫",
    "JaniceAQuoi",
    "EmergRN",
    "Martin (MAH-tin) 🫘",
    "max asher miller"
]

df_eval = dfsample[dfsample["users_display_name"].isin(evaluation_users)][['users_display_name','users_bio','posts_last_2_weeks']]

df_eval

,users_display_name,users_bio,posts_last_2_weeks
0,Kettering Foundation,"Nonpartisan, nonprofit, operating foundation r...","Every year, we seek out extraordinary individu..."
2,William the Mad,"Artist, Creator, and Heresiarch\nHe/She/They/I...",timothy timepiece i miss u… come home catboy |...
3,WOKE_AI_GOD,HE \ HIM,"Yeah Liz, this will totally get you that pardo..."
4,💙🌊 Winn 🌊💙,"Retired, mom/g’mom dem, gardener\nDEMOCRACY, C...","ABSOLUTELY AGREE 💯%....... || ""Sorry, Donnie, ..."
6,MarieGuarde,WORLDPeace 🕊️☮️ PEACE💜\nNoKings! Equal Rights ...,Even the WSJ is sounding the alarm: Billionair...
8,Jeff/傑夫,(he/他)🏳️‍🌈🏳️‍⚧️🇹🇼🇺🇦🌱🐦🌻🍉\n\nFormer hospitality ...,A pro-China influence campaign is using automa...
10,JaniceAQuoi,"Philosophy school dropout, mistress gardener, ...",define a “pajama” || For anyone who doesn't kn...
13,EmergRN,🌈🇺🇸💙🌊🙏 ❤️🇨🇦❤️\n❤️ USA! Fearless! 🇺🇸🦅⚖️ \nDemo...,It’s not enough for Fascists in America to alr...
14,Martin (MAH-tin) 🫘,"Salem, MA resident! Acoustic Murder guitarist....","See you there, AIPAC puppet. || If only you we..."
16,max asher miller,"✡️🏳️‍🌈(🟥🟨🟦)(he/him)\nfiction writer, journalis...",The increasingly dominant narrative on the rig...


In [69]:
def classify_user_debug(bio, posts):
    bio = "" if pd.isna(bio) else str(bio)
    posts = "" if pd.isna(posts) else str(posts)
    prompt = PROMPT_TEMPLATE_1.format(bio=bio, posts=posts)

    resp = client.responses.create(
        model=MODEL,
        input=[{"role": "user", "content": prompt}],
        text={"format": FORMAT},
        reasoning={"effort": "low"},
    )

    print("output_text (first 300):", (resp.output_text or "")[:300])
    return json.loads(resp.output_text)

In [70]:
classify_user_debug(df_eval.loc[14, "users_bio"], df_eval.loc[14, "posts_last_2_weeks"])

output_text (first 300): {"tag":"Progressive-Left","confidence":7}


{'tag': 'Progressive-Left', 'confidence': 7}